In [59]:
import numpy as np
import pandas as pd
from scipy.optimize import fmin

In [60]:
%run optimizer.py

# Importing Data

In [61]:
df = pd.read_csv('data/heart.csv',delimiter=';')
df['oldpeak'] = df['oldpeak'].str.replace(',','.')
df['bias'] = 1

In [62]:
df.head()

,age,trestbps,chol,thalach,oldpeak,"target,",bias
0,63,145,233,150,2.1,1,1
1,37,130,250,187,3.5,1,1
2,41,130,204,172,1.4,1,1
3,56,120,236,178,0.8,1,1
4,57,120,354,163,0.6,1,1


In [63]:
v = df['target,'].values.astype('float')
df.drop(['target,'],axis=1,inplace=True)
u = df.values.astype('float')

# Generating J(x)

In [64]:
def J(x):
    return np.sum([ (1-v[i])*x@u[i,:] + np.log(1+np.exp(- x@u[i,:])) for i in range(v.shape[0]) ])

def dJ(x):
    return np.sum(np.array([ (1-v[i])*u[i,:] -(u[i,:]/(np.exp(x@u[i,:])+1)) for i in range(v.shape[0]) ]),axis=0)

# Comparing optimalization methods

In [65]:
from timeit import default_timer as timer

In [66]:
method_names = ['BFGS - suboptimal step','DFP - suboptimal step','BFGS - optimal step', 'DFP - optimal step']
time_list = []
x_list = []

In [67]:
x0 = np.zeros(6)
x0[0] = -2.28

In [68]:
t = timer()
x_list.append(optimize(x0=x0, f=J, df=dJ, method='BFGS', optimal_step=False))
time_list.append(timer()-t)

<ipython-input-64-97fc82834ffa>:2: RuntimeWarning:

overflow encountered in exp



In [69]:
t = timer()
x_list.append(optimize(x0=x0, f=J, df=dJ, method='DFP', optimal_step=False))
time_list.append(timer()-t)

<ipython-input-64-97fc82834ffa>:2: RuntimeWarning:

overflow encountered in exp



In [70]:
t = timer()
x_list.append(optimize(x0=x0, f=J, df=dJ, method='BFGS', optimal_step=True))
time_list.append(timer()-t)

<ipython-input-64-97fc82834ffa>:5: RuntimeWarning:

overflow encountered in exp



In [71]:
t = timer()
x_list.append(optimize(x0=x0, f=J, df=dJ, method='DFP', optimal_step=True))
time_list.append(timer()-t)

<ipython-input-64-97fc82834ffa>:5: RuntimeWarning:

overflow encountered in exp



In [72]:
methods_df = pd.DataFrame(method_names)
methods_df['time'] = time_list
methods_df = methods_df.rename(columns={0:'name'})
methods_df

,name,time
0,BFGS - suboptimal step,0.328335
1,DFP - suboptimal step,0.289212
2,BFGS - optimal step,1.073282
3,DFP - optimal step,24.165039


In [73]:
import plotly.express as px

In [74]:
fig = px.bar(methods_df,x='name',y='time',color="name")
fig.update_layout(title='Comparing speed of quasi newton optimalization methods')

In [122]:
opt_list = [J(x_list[i][-1]) for i in range(len(method_names))]
diff_opt = [i-min(opt_list)+1e-12 for i in opt_list]

In [124]:
fig = px.bar(x=method_names,y=diff_opt,color=method_names)
fig.update_layout(title='Comparing differences of J* in methods')

In [125]:
iter_list = [len(x_list[i]) for i in range(len(method_names))]

In [127]:
fig = px.bar(x=method_names,y=iter_list,color=method_names)
fig.update_layout(title='Comparing number of itterations',yaxis_title='num of iterations')

# Plot J(x<sup>k</sup>) - J*

In [83]:
def tempF(x):
    temp = np.ones(5)
    if type(x) != type(temp):
        return x
    return J(x)

In [107]:
iter_df = pd.DataFrame(x_list,index=method_names).T
conv_df = pd.DataFrame(iter_df[method_names[0]].apply(tempF))
for i in range(1,len(method_names)):
    conv_df[method_names[i]] = pd.DataFrame(iter_df[method_names[i]].apply(tempF))

for index,row in conv_df.iterrows():
    for i,col in enumerate(row.keys()):
        conv_df.loc[index,col] -= J(x_list[i][-1])

In [108]:
conv_df

,BFGS - suboptimal step,DFP - suboptimal step,BFGS - optimal step,DFP - optimal step
0,16313.958115,16313.958115,1.631396e+04,1.631396e+04
1,3505.945652,3505.945652,1.597675e+06,1.597675e+06
2,2879.180841,2878.856378,inf,inf
3,2799.151710,2807.103025,inf,inf
4,2750.178206,2765.591275,inf,inf
...,...,...,...,...
747,NaN,NaN,NaN,1.053267e-07
748,NaN,NaN,NaN,1.044623e-08
749,NaN,NaN,NaN,4.866649e-10
750,NaN,NaN,NaN,8.782308e-12


In [113]:
fig = px.line(conv_df,log_y=True)
fig.update_layout(title='J(x)-J* for different methods',xaxis_title="iteration",yaxis_title='J(x)-J*')

In [116]:
fig = px.line(conv_df.iloc[:32,:-1],log_y=True)
fig.update_layout(title='J(x)-J* without DFP - optimal step',xaxis_title="iteration",yaxis_title='J(x)-J*')

# Summary:

**Fastest method** : DFP with suboptimal step

**Best optimal value** : DFP with or without optimal step

**Least number of iterations** : BFGS with suboptimal step (DFP with suboptimal step is a close second)


**DFP with suboptimal step** is best for our application